In [3]:
import fml_lib
import numpy as np
import pandas as pd
import sys
import math
from math import sqrt, exp, log
import scipy.optimize as so
import scipy
import sdepy
from ouparams import ouparams
import matplotlib.pyplot as plt
from itertools import product
from random import gauss
from statsmodels.tsa.ar_model import AutoReg
import statsmodels.api as sm

Exercise 1: Suppose you are an execution trader. A client calls you with an order to cover a
short position she entered at a price of 100. She gives you two exit conditions:
profit-taking at 90 and stop-loss at 105.


Q1: Assuming the client believes the price follows an O-U process, are these
levels reasonable? For what parameters?

A: These levels are reasonable for a standard deviation 

In [ ]:
rPT=rSLm=np.linspace(0,10,21)
count=0
for prod_ in product([10,5,0,-5,-10],[5,10,25,50,100]):
    count+=1
    coeffs={'forecast':prod_[0],'hl':prod_[1],'sigma':1.5}
    # define sigma as 1.5
    output=batch(coeffs,nIter=1e5,maxHP=100,rPT=rPT,rSLm=rSLm)


Exercise 2: Fit the time series of dollar bars of E-mini S&P 500 futures to an O-U process

In [4]:
def __compute_log_likelihood(params, *args):
    '''
    Compute the average Log Likelihood, this function will by minimized by scipy.
    Find in (2.2) in linked paper

    returns: the average log likelihood from given parameters
    '''
    # functions passed into scipy's minimize() needs accept one parameter, a tuple of
    #   of values that we adjust to minimize the value we return.
    #   optionally, *args can be passed, which are values we don't change, but still want
    #   to use in our function (e.g. the measured heights in our sample or the value Pi)

    theta, mu, sigma = params
    X, dt = args
    n = len(X)

    sigma_tilde_squared = sigma ** 2 * (1 - exp(-2 * mu * dt)) / 2 * mu

    summation_term = 0

    for i in range(1, len(X)):
        summation_term += (X[i] - X[i - 1] * exp(-mu * dt) - theta * (1 - exp(-mu * dt))) ** 2

    summation_term = -summation_term / (2 * n * sigma_tilde_squared)

    log_likelihood = (-log(2 * math.pi) / 2) + (-log(sqrt(sigma_tilde_squared))) + summation_term

    return -log_likelihood
    # since we want to maximize this total log likelihood, we need to minimize the
    #   negation of the this value (scipy doesn't support maximize)


def estimate_coefficients_MLE(X, dt):
    '''
    Estimates Ornstein-Uhlenbeck coefficients (θ, µ, σ) of the given array
    using the Maximum Likelihood Estimation method

    input: X - array-like data to be fit as an OU process
    returns: θ, µ, σ, Total Log Likelihood
    '''

    bounds = ((0, None), (None, None), (0, None))  # theta > 0, mu ∈ ℝ, sigma > 0
    mu_init = np.mean(X)
    result = so.minimize(__compute_log_likelihood, (1e-6, 1e-6, 1e-6), args=(X, dt), bounds=bounds)
    theta, mu, sigma = result.x
    max_log_likelihood = -result.fun  # undo negation from __compute_log_likelihood
    return theta, mu, sigma, max_log_likelihood

In [6]:
dollar_bars = pd.read_csv("USA500IDXUSD_bars.csv", parse_dates=[['Date', 'Timestamp']])
dollar_bars = dollar_bars.set_index("Date_Timestamp")
close_prices = dollar_bars["Close"].values

C:\Users\dstek\AppData\Local\Temp\ipykernel_40828\2420848124.py:1: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  dollar_bars = pd.read_csv("USA500IDXUSD_bars.csv", parse_dates=[['Date', 'Timestamp']])


In [7]:
x = close_prices[:-1]
y = close_prices[1:]
close_prices = np.array(close_prices)
close_time = np.array(dollar_bars.index.values)
sm.add_constant(x)
ar_sim = AutoReg(y,1).fit()
mu,sigma, theta = estimate_coefficients_MLE(close_prices, close_time[1]-close_time[0])



: 

In [41]:
print(ar_sim.params)
print([mu, sigma, theta])
print(close_prices)

[0.00112643 0.99999986]
[8032.981292866822, 0.42481407396950527, 1.4104779950638593e-07]
float64
[4447.626 4448.427 4448.111 ... 4868.626 4868.361 4868.376]
